<a href="https://colab.research.google.com/github/imhyunho99/2023-1--Deaplearning_Framework/blob/main/(PR)1_Char_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Simple Character LSTM
# Char RNN에서 설명한 부분은 생략했습니다.

import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [12]:
# Preprocessing string data
# alphabet(0-25), space(26),..., start, end 

string = "hello pytorch. how long can a rnn cell remember? show me your limit! limit limit limit"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
char_len = len(char_list)

char_len

35

In [13]:
# String to onehot vector
# [0 0 0 ... 1 0] -> Start
# [1 0 0 ... 0 0] -> a 
# [0 1 0 ... 0 0] -> b
# [0 0 1 ... 0 0] -> c
# [0 0 0 ... 0 1] -> end

def string_to_onehot(string):
    start = np.zeros(shape=char_len ,dtype=int)
    end = np.zeros(shape=char_len ,dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape=char_len ,dtype=int)
        zero[idx]=1
        start = np.vstack([start,zero])
    output = np.vstack([start,end])
    return output

In [14]:
# Onehot vector to word


def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [15]:
# 하이퍼파라미터 설정
# 문자열을 단어 하나씩 잘러서 사용하는걸로 구현해서 batch_size 1로 고정입니다.
# batch_size가 1보다 큰 경우는 다음 실습코드에 있습니다.
batch_size = 1

# seq_len는 바꿔도 학습은 되지만 테스트시 편의성을 위해 1로 설정했습니다.
seq_len = 1

# num_layers는 입력 형식에만 맞게 형태를 바꿔주면 됩니다.
num_layers = 3
input_size = char_len#35
hidden_size = 35 
lr = 0.01
num_epochs = 1000

one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

print(one_hot.size())

torch.Size([88, 35])


In [16]:
# RNN with 1 hidden layer

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers)
        
    def forward(self,input_,hidden,cell):
        output,(hidden,cell) = self.lstm(input_,(hidden,cell))#tuple(hidden,cell)
        return output,hidden,cell
    
    def init_hidden_cell(self):
        hidden = torch.zeros(num_layers,batch_size,hidden_size)
        cell = torch.zeros(num_layers,batch_size,hidden_size)
        return hidden,cell
    
rnn = RNN(input_size,hidden_size, num_layers)

In [17]:
# Loss function & Optimizer

loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [18]:
j=0
input_data = one_hot[j:j+seq_len].view(seq_len, batch_size, input_size)
print(input_data.size())

hidden,cell = rnn.init_hidden_cell()
print(hidden.size(),cell.size())

output, hidden,cell = rnn(input_data,hidden,cell)
print(output.size(),hidden.size(),cell.size())

torch.Size([1, 1, 35])
torch.Size([3, 1, 35]) torch.Size([3, 1, 35])
torch.Size([1, 1, 35]) torch.Size([3, 1, 35]) torch.Size([3, 1, 35])


In [19]:
one_hot[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [20]:
unroll_len = one_hot.size()[0]//seq_len -1
for i in range(num_epochs):
    hidden,cell = rnn.init_hidden_cell()
    
    loss = 0
    for j in range(unroll_len):
        input_data = one_hot[j:j+seq_len].view(seq_len,batch_size,input_size) 
        label = one_hot[j+1:j+seq_len+1].view(seq_len,batch_size,input_size)
        
        optimizer.zero_grad()
        
        output, hidden, cell = rnn(input_data,hidden,cell)
        loss += loss_func(output.view(1,-1),label.view(1,-1))
        
    loss.backward()
    optimizer.step()

    if i%10 ==0:
        print(loss)

tensor(2.8502, grad_fn=<AddBackward0>)
tensor(2.2782, grad_fn=<AddBackward0>)
tensor(2.1866, grad_fn=<AddBackward0>)
tensor(2.0682, grad_fn=<AddBackward0>)
tensor(1.7588, grad_fn=<AddBackward0>)
tensor(1.3466, grad_fn=<AddBackward0>)
tensor(0.8482, grad_fn=<AddBackward0>)
tensor(0.4885, grad_fn=<AddBackward0>)
tensor(0.2602, grad_fn=<AddBackward0>)
tensor(0.1482, grad_fn=<AddBackward0>)
tensor(0.0827, grad_fn=<AddBackward0>)
tensor(0.0504, grad_fn=<AddBackward0>)
tensor(0.0363, grad_fn=<AddBackward0>)
tensor(0.0280, grad_fn=<AddBackward0>)
tensor(0.0224, grad_fn=<AddBackward0>)
tensor(0.0189, grad_fn=<AddBackward0>)
tensor(0.0164, grad_fn=<AddBackward0>)
tensor(0.0155, grad_fn=<AddBackward0>)
tensor(0.0134, grad_fn=<AddBackward0>)
tensor(0.0119, grad_fn=<AddBackward0>)
tensor(0.0103, grad_fn=<AddBackward0>)
tensor(0.0096, grad_fn=<AddBackward0>)
tensor(0.0085, grad_fn=<AddBackward0>)
tensor(0.0078, grad_fn=<AddBackward0>)
tensor(0.0073, grad_fn=<AddBackward0>)
tensor(0.0069, grad_fn=<A

In [21]:
hidden,cell = rnn.init_hidden_cell()

for j in range(unroll_len-1):
    input_data = one_hot[j:j+1].view(1,batch_size,hidden_size) 
    label = one_hot[j+1:j+1+1].view(1,batch_size,hidden_size) 
    
    output, hidden, cell = rnn(input_data,hidden,cell)
    print(onehot_to_word(output.data),end="") 

hello pytorch. how long can a rnn cell remember? show me your limit! limit limit limit